In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets #CIFAR10, CIFAR100
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
!mkdir '/data'

In [4]:
def data_loader(data_dir, batch_size, random_seed = 42, shuffle = True, valid_size=0.1, test = False):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean = (0.4914, 0.4822, 0.4465),
            std = (0.2023, 0.1994, 0.2010)
        )
    ])

    if test:
        dataset = datasets.CIFAR100(
            root = data_dir,
            train = False,
            download = True,
            transform = transform
        )

        data_loader = DataLoader(
            dataset, batch_size = batch_size, shuffle = shuffle
        )

        return data_loader

    train_dataset = datasets.CIFAR100(
        root = data_dir,
        train = True,
        download = True,
        transform = transform
    )

    valid_dataset = datasets.CIFAR10(
        root = data_dir,
        train = True,
        download = True,
        transform = transform
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size*num_train))

    if shuffle:
      np.random.seed(random_seed)
      np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = DataLoader(
       train_dataset, batch_size=batch_size, sampler=train_sampler
    )

    valid_loader = DataLoader(
       valid_dataset, batch_size=batch_size, sampler=valid_sampler
    )

    return (train_loader, valid_loader)


train_loader, valid_loader = data_loader(data_dir='./data',
                                         batch_size=64)

test_loader = data_loader(data_dir='./data',
                              batch_size=64,
                              test=True)

100%|██████████| 169001437/169001437 [00:02<00:00, 62189583.55it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


100%|██████████| 170498071/170498071 [00:02<00:00, 73887238.51it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
class VGG16(nn.Module):
    def __init__ (self, num_classes = 10):
        super(VGG16, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )

        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )

        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )

        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU()
        )

        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU()
        )

        self.fc2 = nn.Sequential(
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)

        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)

        return out

In [6]:
def train(num_epochs, train_loader, valid_loader, classifier, criterion, optimizer, device):
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            #Forward Pass
            outputs = classifier(images)
            loss = criterion(outputs, labels)

            #Back Propagation and optimizers
            optimizer.zero_grad()  #W_new = W_old - LR * Gradient
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch+1, num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        with torch.no_grad():
            correct = 0
            total = 0

            for images, labels in valid_loader:
                images = images.to(device)
                labels = labels.to(device)

                outputs = classifier(images)

                _, predicted = torch.max(outputs.data, 1)

                total += labels.size(0)

                correct +=(predicted == labels).sum().item()
                del images, labels, outputs

            print(f'Accuracy of the model on validation data: {(correct/total)*100}%')

In [7]:
def evaluate(test_loader, classifier, criterion, device):
    classifier.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        total_loss = 0.0

        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = classifier(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            total_loss += loss.item()

        accuracy = (correct / total) * 100
        average_loss = total_loss / total

        print(f'Accuracy of the model on test data: {accuracy:.2f}%')
        print(f'Average loss of the model on test data: {average_loss:.4f}')

In [8]:
num_classes = 100
num_epochs = 20
batch_size = 16
learning_rate = 0.005

vgg16_classifier = VGG16(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(vgg16_classifier.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

In [9]:
train(num_epochs, train_loader, valid_loader, vgg16_classifier, criterion, optimizer, device)

Epoch [(1, 20)], Step [704/704], Loss: 3.8030
Accuracy of the model on validation data: 0.88%
Epoch [(2, 20)], Step [704/704], Loss: 2.5507
Accuracy of the model on validation data: 0.96%
Epoch [(3, 20)], Step [704/704], Loss: 4.1271
Accuracy of the model on validation data: 1.16%
Epoch [(4, 20)], Step [704/704], Loss: 1.4106
Accuracy of the model on validation data: 0.72%
Epoch [(5, 20)], Step [704/704], Loss: 1.3709
Accuracy of the model on validation data: 1.08%
Epoch [(6, 20)], Step [704/704], Loss: 1.0132
Accuracy of the model on validation data: 0.72%
Epoch [(7, 20)], Step [704/704], Loss: 2.5442
Accuracy of the model on validation data: 0.5%
Epoch [(8, 20)], Step [704/704], Loss: 1.6641
Accuracy of the model on validation data: 1.2%
Epoch [(9, 20)], Step [704/704], Loss: 3.1967
Accuracy of the model on validation data: 0.66%
Epoch [(10, 20)], Step [704/704], Loss: 1.1313
Accuracy of the model on validation data: 1.1199999999999999%
Epoch [(11, 20)], Step [704/704], Loss: 1.2740


In [10]:
evaluate(test_loader, vgg16_classifier, criterion, device)

Accuracy of the model on test data: 56.62%
Average loss of the model on test data: 0.0247
